# Data Analysis

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import geocoder
from opencage.geocoder import OpenCageGeocode
import folium
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Scraping data

Using the pandas library to read the html

In [2]:
wiki_data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
wiki_data.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


## Cleaning the data

- Removing the not assigned borough
- Replacing the '/' with a ',' where we have multiple entries for Neighborhood
- There are no enteries where we donot have a value for Neighborhood

In [3]:
neighborhood_data = pd.DataFrame(columns = ['Postal code','Borough', 'Neighborhood'], 
                                 data = wiki_data[wiki_data.Borough!='Not assigned'].values)
neighborhood_data.Neighborhood = neighborhood_data.Neighborhood.replace({'/':','},regex=True)
neighborhood_data.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [4]:
print("The size of the data frame read is {}".format(neighborhood_data.shape))

The size of the data frame read is (103, 3)


## Fetching the latitude and longitude

Using geocoder package to get the latitude and longitude
- But this failed so used the csv file provided.

In [5]:
# for i,data in neighborhood_data.iterrows():
#     lat_lng = None
#     while(lat_lng==None):
#         location = geocoder.google('{}, Toronto, Ontario'.format(data['Postal code']))
#         lat_lng = location.latlng
#     neighborhood_data['Latitude'] = lat_lng[0]
#     neighborhood_data['Longitude'] = lat_lng[1]
#     print(i, data['Postal code'])
#     time.sleep(1.0)

### Reading the coordinates data

The index of the data frame is set to be the Postal code column.

In [6]:
lat_lng = pd.read_csv('./Geospatial_Coordinates.csv')
lat_lng.set_index('Postal Code', inplace=True)
lat_lng.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


## Merging the latitude and longitude data with neighborhood data

In [7]:
neighborhood_data['Latitude'] = lat_lng.loc[neighborhood_data['Postal code'], 'Latitude'].values
neighborhood_data['Longitude'] = lat_lng.loc[neighborhood_data['Postal code'], 'Longitude'].values
neighborhood_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


## Analysing and understanding the data

Let's check how many unique Borough's and Neighborhood's are present.

In [8]:
print('The data has {} boroughs and {} neighborhoods.'.format(
        len(neighborhood_data['Borough'].unique()),
        len(neighborhood_data['Neighborhood'].unique())
    )
)

The data has 10 boroughs and 98 neighborhoods.


## Getting the latitude and longitude of the Toronto 

Geocoder package was not working so used open cage geocode

In [9]:
# lat_lng = None
# while(lat_lng==None):
#     location = geocoder.google('Toronto, Ontario')
#     lat_lng = location.latlng
#  = location.latlng[0]
#  = location.latlng[1]
geolocator = OpenCageGeocode("d9151bc21e3c47daa72ac27bf5422c76")
location = geolocator.geocode("Toronto", country="CA")
latitude_TO = location[0]['geometry']['lat']
longitude_TO = location[0]['geometry']['lng']
print("The latitude and longitude of Toronto, CA are {}, {}".format(latitude_TO, longitude_TO))


The latitude and longitude of Toronto, CA are 43.6534817, -79.3839347


## Plotting the neighborhoods on a Map

In [10]:
map_toronto = folium.Map(location=[latitude_TO, longitude_TO], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_data['Latitude'], neighborhood_data['Longitude'], neighborhood_data['Borough'], neighborhood_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Declaring foursquare api 

In [11]:
CLIENT_ID = 'UFX0WT0O50CEQBP4GIIDCPB5FELMNMCMDN50MAJGB0GDZDBV' # your Foursquare ID
CLIENT_SECRET = 'WCNIIINW5HL1SHJBEUSBIRA5FC35GB4BHYAPN2UYCHWYFPRF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UFX0WT0O50CEQBP4GIIDCPB5FELMNMCMDN50MAJGB0GDZDBV
CLIENT_SECRET:WCNIIINW5HL1SHJBEUSBIRA5FC35GB4BHYAPN2UYCHWYFPRF


## Explore the Neighborhoods

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Venues withing 500m radius and top 100

In [ ]:
toronto_venues = getNearbyVenues(names=neighborhood_data['Neighborhood'],
                                   latitudes=neighborhood_data['Latitude'],
                                   longitudes=neighborhood_data['Longitude']
                                  )
toronto_venues.head()

### The size of the data frame with venues

In [ ]:
print(toronto_venues.shape)

### The number of unique categories from each venue

In [ ]:
print("The number of unique categories are {}".format(len(toronto_venues['Venue Category'].unique())))

### One hot encoding

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# # move neighborhood column to the first column
neighborhoodIndex = list(toronto_onehot.columns).index('Neighborhood')
fixed_columns = [toronto_onehot.columns[neighborhoodIndex]] + list(toronto_onehot.columns[:neighborhoodIndex])+list(toronto_onehot.columns[neighborhoodIndex+1:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
print("The size of the onehot dataframe is {}".format(toronto_onehot.shape))

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
print("The new size is {}".format(toronto_grouped.shape))

### Top 10 common venues

In [ ]:
# num_top_venues = 10

# for hood in toronto_grouped['Neighborhood']:
#     print("----"+hood+"----")
#     temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
#     temp.columns = ['venue','freq']
#     temp = temp.iloc[1:]
#     temp['freq'] = temp['freq'].astype(float)
#     temp = temp.round({'freq': 2})
#     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#     print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Clustering the Neighborhoods

Using k-means clustering

In [ ]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

### Including cluster and the most common venues in data frame

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhood_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

In [ ]:
toronto_merged = toronto_merged['Cluster Labels'].dropna(inplace=True)

### Visualizing the clusters

In [ ]:
map_clusters = folium.Map(location=[latitude_TO, longitude_TO], zoom_start=10)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

## Analysing the Clusters

In [ ]:
toronto_merged.reset_index(inplace=True)

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]